In [1]:
import sys
sys.path[0]+='/..'
# from graph_dataset.dataset import GraphDataset
from dataset import GraphDataset

from model.gcn_gat import GNNPolicy
from utilities import *
import torch
import torch_geometric
import os 
import time
from torch.optim.lr_scheduler import CosineAnnealingLR
from readdataUC import UC
import gurobipy as gp
import json


In [2]:
#set folder
uc_model = '3bin'
instance = '132_c11_based_8_std'
model = 'gat'

model_save_path = f'../model_save/{uc_model}_{instance}_{model}/'
log_save_path =   f"../model_save/{uc_model}_{instance}_{model}_log/"
if not os.path.isdir(model_save_path):
    os.mkdir(model_save_path)
if not os.path.isdir(log_save_path):
    os.mkdir(log_save_path)

log_file = open(f'{log_save_path}train.log', 'wb')

In [3]:
LEARNING_RATE = 1e-4
NB_EPOCHS =2000
BATCH_SIZE = 1
NUM_WORKERS = 2
ETA_MIN=1e-5

In [4]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PredictModel = GNNPolicy().to(DEVICE)
# if os.path.exists(model_save_path+'model_best.pth'):
#     state_dict = torch.load(model_save_path+'model_best.pth')
#     PredictModel.load_state_dict(state_dict)
#     print("载入模型成功")

In [5]:
train_data = GraphDataset(f'../datasets/{instance}/{uc_model}/train/')[::30]
valid_data = GraphDataset(f'../datasets/{instance}/{uc_model}/valid/')[::30]
# train_data = GraphDataset(f'../datasets/mixed/train/')
# valid_data = GraphDataset(f'../datasets/mixed/valid/')

train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [6]:
file = '../UC_AF/'+instance+'.mod'

dataUC=UC(file)

N = dataUC.NumThermal
T = dataUC.HorizonLen
source_node=[]
target_node=[]
for i in range(N):
    for t in range(T):
        j=i*T+t
        source_node.append(j)
        target_node.append(j)
        source_node.append(j+N*T)
        target_node.append(j)
        source_node.append(j+2*N*T)
        target_node.append(j)
        source_node.append(j+3*N*T)
        target_node.append(j)
        source_node.append(j+4*N*T)
        target_node.append(j)
        # target_node.append(j+5*N*T)

        for ii in range(N):
            if ii != i:
                source_node.append(ii*T+t)
                target_node.append(j)
        for tt in range(T):
            if tt != t:
                source_node.append(i*T+tt)
                target_node.append(j)
edge_index=torch.tensor([source_node,target_node], dtype=torch.long).to(DEVICE)
print(len(source_node))

503712


In [7]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    loss_fn = torch.nn.BCELoss()

    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):
            batch = batch.to(DEVICE)
            weighted_sol = batch.weighted_sol
            #predict the binary distribution, BD
            BD = predict(
                batch.constraint_features,
                batch.edge_index,
                batch.edge_attr,
                batch.variable_features,
                edge_index
            )
            # compute loss
            # print(batch.constraint_features.shape,batch.variable_features.shape)
            presol = BD[batch.b_vars==1]
            presol = presol.sigmoid()
            loss = loss_fn(presol, weighted_sol)
            
            

            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed

    return mean_loss


In [8]:
def ps_accelerate(predict,file):
    
    with gp.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.setParam('MIPGap', 1e-3)
        env.start()
        m = gp.read(file,env)
        f = m.getObjective()
        k=1
        A_indices, A_values, v_nodes, c_nodes, b_vars=get_fea(m)
        
        sol = predict(c_nodes.to(DEVICE), A_indices.to(DEVICE), A_values.to(DEVICE), v_nodes.to(DEVICE), edge_index).sigmoid()  

        sol = sol[b_vars==1]
        sol = he_fs(m,sol,k,instance)
        sol = remove_initial(sol,m)
        # print(file.split('/')[5].split('_')[0])
        day = int(file.split('/')[5].split('_')[0])
        sol = restore_on_off(sol,instance,day)
        m = fix_variable(m,sol)

        m.Params.TimeLimit = 10
        m.optimize()
        if m.Status==2 or m.Status ==9:
            return f.getValue()
        else:
            return -1

In [9]:
def valid(predict):
    lp_path=f'../datasets/{instance}/lp/valid/'
    with open(f'../datasets/{instance}/lp/valid/obj.json', 'r') as f:
        objs = json.load(f)
    inf_cnt=0
    opt_loss=[]
    for file in os.listdir(lp_path):
        if file.endswith('.lp'):
            lp_file = os.path.join(lp_path, file)
            obj = ps_accelerate(predict,lp_file)
            if obj == -1:
                inf_cnt+=1
            else:
                opt_loss.append(obj/objs[file.split('.')[0]])
    return sum(opt_loss)/len(opt_loss),inf_cnt/(len(opt_loss)+inf_cnt)
# valid(PredictModel)

In [10]:
optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=LEARNING_RATE,weight_decay=1e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS/10, eta_min=ETA_MIN)
best_val_loss = 0.2

In [11]:
cur_epoch = 0
for epoch in range(NB_EPOCHS):
    begin=time.time()
    cur_epoch+=1
    train_loss = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.6f}",end='\t')
    valid_loss = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.6f}")
    acc,infs=valid(PredictModel)
    print(acc)
    if acc<best_acc:
        cur_epoch=0
        best_acc = acc
        print(best_acc)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
    torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
    #100个epochloss不下降就停止
    if cur_epoch>100 and valid_loss>best_val_loss:
        break
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

Epoch 0 Train loss: 0.606247	Epoch 0 Valid loss: 0.511849
(1.034201816692129, 0.00919963201471941)
Epoch 1 Train loss: 0.485226	Epoch 1 Valid loss: 0.300666
(1.031302658993964, 0.019319227230910764)
Epoch 2 Train loss: 0.271899	Epoch 2 Valid loss: 0.340288
(1.0315445608402807, 0.00919963201471941)
Epoch 3 Train loss: 0.223515	Epoch 3 Valid loss: 0.165964
(1.028519217933438, 0.022999080036798528)
0.16596400838445974
Epoch 4 Train loss: 0.191626	Epoch 4 Valid loss: 0.167704
(1.0279372252289927, 0.017479300827966882)
Epoch 5 Train loss: 0.164586	Epoch 5 Valid loss: 0.156321
(1.027395735451435, 0.00919963201471941)
0.1563210698801118
Epoch 6 Train loss: 0.167427	Epoch 6 Valid loss: 0.155919
(1.0284206099229958, 0.00919963201471941)
0.15591931967316447
Epoch 7 Train loss: 0.156961	Epoch 7 Valid loss: 0.220394
(1.0278757689080633, 0.01839926402943882)
Epoch 8 Train loss: 0.159118	Epoch 8 Valid loss: 0.167360
(1.0279933591198138, 0.01839926402943882)
Epoch 9 Train loss: 0.154619	Epoch 9 Valid

KeyboardInterrupt: 